In [1]:
import torch
from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.utils import register_all_modules
import os

DEBUG_MODE = False  # Set to True for debugging mode
print("="*50)
print(f"      MODE DEBUGGING: {'AKTIF' if DEBUG_MODE else 'NONAKTIF'}")
print("="*50)

# ===== Step 1: Setup & Registrasi =====
register_all_modules()

print("--- Tes Ketersediaan GPU PyTorch ---")
print(f"Apakah CUDA tersedia? -> {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU terdeteksi: {torch.cuda.get_device_name(0)}")
else:
    print("PERINGATAN: Tidak ada GPU terdeteksi. Training akan berjalan di CPU dan akan sangat lambat.")

      MODE DEBUGGING: NONAKTIF
--- Tes Ketersediaan GPU PyTorch ---
Apakah CUDA tersedia? -> True
GPU terdeteksi: NVIDIA GeForce RTX 5060


In [ ]:
# ===== Step 2: Load Konfigurasi Model (RetinaNet) =====
print("--- INFO: Memilih model backbone Swin Transformer ---")
#untuk retinanet
cfg_path = 'mmdetection/configs/swin/retinanet_swin-t-p4-w7_fpn_1x_coco.py' 
#untuk faster-rcnn
# cfg_path = 'mmdetection/configs/swin/faster-rcnn_swin-t-p4-w7_fpn_1x_coco.py'
cfg = Config.fromfile(cfg_path)

# Ambil nama model dari path config
model_name = None
if "retinanet" in cfg_path.lower():
    model_name = "retinanet"
elif "faster-rcnn" in cfg_path.lower():
    model_name = "fasterrcnn"
else:
    model_name = "custommodel"

print(f"--- INFO: Model yang digunakan: {model_name} ---")

# ===== [FIX FINAL] Definisikan Ulang Pipeline dengan Benar =====
print("--- INFO: Mendefinisikan ulang pipeline untuk train dan validasi ---")

# Definisikan pipeline untuk training
# Kita pastikan 'LoadAnnotations' tidak memuat mask
train_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=False), # Pastikan with_mask=False
    dict(type='Resize', scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', prob=0.5),
     dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],   # mean ImageNet
        std=[58.395, 57.12, 57.375],      # std ImageNet
        to_rgb=True
    ),
    dict(type='PackDetInputs'),
]

# Definisikan pipeline untuk validasi dan tes
# KUNCI UTAMA ADA DI SINI: pastikan 'scale_factor' ada di 'meta_keys'
val_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='Resize', scale=(1333, 800), keep_ratio=True),
    # `LoadAnnotations` dibutuhkan untuk evaluasi
    dict(type='LoadAnnotations', with_bbox=True, with_mask=False), # Pastikan with_mask=False
    dict(
        type='PackDetInputs',
        meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
                   'scale_factor')) # PASTIKAN 'scale_factor' ADA DI SINI
]

# Terapkan pipeline yang sudah didefinisikan
cfg.train_dataloader.dataset.pipeline = train_pipeline
cfg.val_dataloader.dataset.pipeline = val_pipeline
cfg.test_dataloader.dataset.pipeline = val_pipeline

--- INFO: Memilih model backbone Swin Transformer ---
--- INFO: Model yang digunakan: retinanet ---
--- INFO: Mendefinisikan ulang pipeline untuk train dan validasi ---


In [3]:
# ===== Step 3: Modifikasi Konfigurasi Dataset (dengan Logika Debug) =====
data_root = 'Dataset/Deepfish_COCO/'
metainfo = {'classes': ('Fish',), 'palette': [(220, 20, 60)]}

# Pilih file anotasi berdasarkan DEBUG_MODE
train_ann_file = 'annotations/mini_instances_train.json' if DEBUG_MODE else 'annotations/instances_train.json'
val_ann_file = 'annotations/mini_instances_valid.json' if DEBUG_MODE else 'annotations/instances_valid.json'
print(f"--- INFO: Menggunakan file anotasi training: {train_ann_file}")
print(f"--- INFO: Menggunakan file anotasi validasi: {val_ann_file}")

# Terapkan konfigurasi dataset
cfg.train_dataloader.dataset.update(dict(
    data_root=data_root, metainfo=metainfo, ann_file=train_ann_file,
    data_prefix=dict(img='train/'), filter_cfg=dict(filter_empty_gt=True)))
cfg.val_dataloader.dataset.update(dict(
    data_root=data_root, metainfo=metainfo, ann_file=val_ann_file,
    data_prefix=dict(img='valid/'), filter_cfg=dict(filter_empty_gt=True)))
cfg.test_dataloader.dataset.update(dict(
    data_root=data_root, metainfo=metainfo, ann_file=val_ann_file,
    data_prefix=dict(img='valid/'), filter_cfg=dict(filter_empty_gt=False)))

--- INFO: Menggunakan file anotasi training: annotations/instances_train.json
--- INFO: Menggunakan file anotasi validasi: annotations/instances_valid.json


In [4]:
# ===== Step 4: Modifikasi Evaluator COCO =====
cfg.val_evaluator.ann_file = data_root + val_ann_file
cfg.test_evaluator.ann_file = data_root + val_ann_file
cfg.val_evaluator.metric = 'bbox'
cfg.test_evaluator.metric = 'bbox'

In [5]:
# ===== Step 5: Atur Kepala Model Deteksi (Model Head) =====
# Untuk RetinaNet, head-nya berbeda dari Faster R-CNN
# cfg.model.bbox_head.num_classes = 1



if hasattr(cfg.model, 'bbox_head'):
    cfg.model.bbox_head.num_classes = 1
elif hasattr(cfg.model, 'roi_head'):
    if isinstance(cfg.model.roi_head.bbox_head, list):
        for head in cfg.model.roi_head.bbox_head:
            head.num_classes = 1
    else:
        cfg.model.roi_head.bbox_head.num_classes = 1


In [ ]:
# ===== Step 6: Atur Pengaturan Training (Training Settings) =====

cfg.optim_wrapper.optimizer.lr = 0.0001
print(f"--- INFO: Learning rate diatur secara eksplisit ke {cfg.optim_wrapper.optimizer.lr} ---")

if not DEBUG_MODE: # Hanya resume jika tidak dalam mode debug
    checkpoint_file = './outputs_swin_retinanet_deepfish/epoch_10.pth' # atau latest.pth
    if os.path.exists(checkpoint_file):
        print(f"--- INFO: Menemukan checkpoint. Akan melanjutkan training dari {checkpoint_file} ---")
        cfg.load_from = checkpoint_file
        cfg.resume = True # <-- INI DIA TEMPATNYA
    else:
        # Jika tidak ada checkpoint, pastikan resume False (default)
        print("--- INFO: Tidak ada checkpoint ditemukan. Memulai training dari awal. ---")
        cfg.resume = False
# ----------------------------------------------------


if DEBUG_MODE:
    cfg.train_cfg.max_epochs = 10
    cfg.default_hooks.logger.interval = 5
    cfg.default_hooks.checkpoint.interval = 1
    print("--- INFO: Pengaturan training dipercepat untuk mode debug.")
else:
    cfg.train_cfg.max_epochs = 15 # Atau lebih
    cfg.default_hooks.logger.interval = 10
    cfg.default_hooks.checkpoint.interval = 1

# Pengaturan checkpoint lainnya
cfg.default_hooks.checkpoint.max_keep_ckpts = 3
cfg.default_hooks.checkpoint.save_best = 'coco/bbox_mAP'
cfg.default_hooks.checkpoint.rule = 'greater'
cfg.visualizer.vis_backends = [
    dict(type='LocalVisBackend'),
    dict(type='TensorboardVisBackend')
]

--- INFO: Learning rate diatur secara eksplisit ke 0.0001 ---
--- INFO: Tidak ada checkpoint ditemukan. Memulai training dari awal. ---


In [7]:
# ===== Step 7: Tentukan Direktori Output =====
# ===== Step 7: Tentukan Direktori Output (Otomatis sesuai model) =====
work_dir_base = f'./outputs_swin_{model_name}_deepfish'
cfg.work_dir = f"{work_dir_base}_debug" if DEBUG_MODE else work_dir_base
print(f"--- INFO: Hasil akan disimpan di folder: {cfg.work_dir} ---")
os.makedirs(cfg.work_dir, exist_ok=True)


--- INFO: Hasil akan disimpan di folder: ./outputs_swin_retinanet_deepfish ---


In [8]:
# ===== Step 8: Verifikasi Dataset Final (Sanity Check) =====
from mmengine.registry import DATASETS

print("\n--- Pengecekan Dataset Final ---")
try:
    train_dataset = DATASETS.build(cfg.train_dataloader.dataset)
    print(f"Jumlah data training yang akan digunakan: {len(train_dataset)}")
    val_dataset = DATASETS.build(cfg.val_dataloader.dataset)
    print(f"Jumlah data validasi yang akan digunakan: {len(val_dataset)}")
except Exception as e:
    print(f"\nFATAL ERROR saat memuat dataset: {e}")
    # Gunakan `raise` di notebook agar error terlihat jelas
    raise e
print("---------------------------------\n")

print("Konfigurasi selesai. Siap untuk menjalankan training di sel berikutnya.")


--- Pengecekan Dataset Final ---
loading annotations into memory...
Done (t=0.08s)
creating index...
index created!
Jumlah data training yang akan digunakan: 3596
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Jumlah data validasi yang akan digunakan: 909
---------------------------------

Konfigurasi selesai. Siap untuk menjalankan training di sel berikutnya.


In [9]:
# ===== Step 9: Mulai Training =====
print(">>> Memulai proses training...")
runner = Runner.from_cfg(cfg)
runner.train()
print(">>> Proses training selesai.")

>>> Memulai proses training...
08/26 18:23:27 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: win32
    Python: 3.10.18 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:08:55) [MSC v.1929 64 bit (AMD64)]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 111036196
    GPU 0: NVIDIA GeForce RTX 5060
    CUDA_HOME: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8
    NVCC: Cuda compilation tools, release 12.8, V12.8.61
    MSVC: n/a, reason: fileno
    PyTorch: 2.8.0+cu128
    PyTorch compiling details: PyTorch built with:
  - C++ Version: 201703
  - MSVC 193833145
  - Intel(R) oneAPI Math Kernel Library Version 2025.2-Product Build 20250620 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.7.1 (Git Hash 8d263e693366ef8db40acc569cc7d8edf644556d)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 12.8
  - N

In [10]:
from mmdet.apis import inference_detector, init_detector
from mmdet.visualization import DetLocalVisualizer
import mmcv
import numpy as np

# --- [PENTING] Pilih Checkpoint Terbaik ---
best_checkpoint_name = None
if os.path.exists(cfg.work_dir):
    best_ckpts = [f for f in os.listdir(cfg.work_dir) if f.startswith('best_') and f.endswith('.pth')]
    if best_ckpts:
        best_checkpoint_name = best_ckpts[0]

if best_checkpoint_name:
    checkpoint_path = os.path.join(cfg.work_dir, best_checkpoint_name)
else:
    checkpoint_path = os.path.join(cfg.work_dir, "epoch_18.pth")  # fallback

print(f"--- INFO: Menggunakan checkpoint: {checkpoint_path} ---")

if not os.path.exists(checkpoint_path):
    print(f"ERROR: File checkpoint tidak ditemukan di '{checkpoint_path}'")
else:
    # Patch kompatibilitas untuk load checkpoint
    orig_load = torch.load
    def torch_load_wrapper(*args, **kwargs):
        kwargs["weights_only"] = False
        return orig_load(*args, **kwargs)
    torch.load = torch_load_wrapper

    # Inisialisasi model
    model = init_detector(cfg, checkpoint_path, device='cuda' if torch.cuda.is_available() else 'cpu')
    
    # Inisialisasi Visualizer
    vis_save_dir = os.path.join(cfg.work_dir, 'vis_outputs')
    os.makedirs(vis_save_dir, exist_ok=True)
    visualizer = DetLocalVisualizer(
        vis_backends=[dict(type='LocalVisBackend')],
        name='visualizer',
        save_dir=vis_save_dir
    )
    visualizer.dataset_meta = cfg.train_dataloader.dataset.metainfo

    # Path gambar uji
    img_path = 'dataset_final/test/dataset2_test__f3fc24dc6fe2db1c604d2eca5a5073e8.jpg'

    if not os.path.exists(img_path):
        print(f"ERROR: Gambar uji tidak ditemukan di '{img_path}'")
    else:
        # Inference
        result = inference_detector(model, img_path)

        # Ambil jumlah ikan terdeteksi berdasarkan threshold
        pred_instances = result.pred_instances
        scores = pred_instances.scores.cpu().numpy()
        labels = pred_instances.labels.cpu().numpy()

        score_thr = 0.3
        keep = scores > score_thr
        num_detected = np.sum(keep)

        print(f">>> Summary Deteksi: Total ikan terdeteksi = {num_detected}")

        # Load gambar
        img = mmcv.imread(img_path)
        img = mmcv.imconvert(img, 'bgr', 'rgb')

        # Visualisasi hasil dengan threshold
        out_file = os.path.join(vis_save_dir, "deteksi_result.jpg")
        visualizer.add_datasample(
            name='result',
            image=img,
            data_sample=result,
            draw_gt=False,
            show=True,            # tampilkan popup
            wait_time=0,
            pred_score_thr=score_thr,
            out_file=out_file     # simpan gambar hasil
        )

        print(f">>> Gambar hasil deteksi disimpan di: {out_file}")


--- INFO: Menggunakan checkpoint: ./outputs_swin_retinanet_deepfish\best_coco_bbox_mAP_epoch_1.pth ---
Loads checkpoint by local backend from path: ./outputs_swin_retinanet_deepfish\best_coco_bbox_mAP_epoch_1.pth
>>> Summary Deteksi: Total ikan terdeteksi = 0
>>> Gambar hasil deteksi disimpan di: ./outputs_swin_retinanet_deepfish\vis_outputs\deteksi_result.jpg
